In [1]:
%env THEANO_FLAGS=device=gpu0,lib.cnmem=0.9,floatX=float32
%env USER_EMB_SIZE=256
import sys

env: THEANO_FLAGS=device=gpu0,lib.cnmem=0.9,floatX=float32
env: USER_EMB_SIZE=256


In [2]:
sys.path.insert(0, '../')

In [3]:
from mymodule.neural.seq2seq import *
from mymodule.base_stuff import Vocab, phrase2matrix
from mymodule.neural.discriminator import DssmModel
from agentnet.utils.persistence import load, save
import codecs
from __future__ import unicode_literals
from copy import deepcopy
from tqdm import tqdm

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5110)


In [4]:
vocab_path = '../twitter_data/tokens.txt'
vocab = Vocab.read_from_file(vocab_path)
dssm_model = DssmModel(vocab, 1000)
load(dssm_model.l_user_semantic, '../weights/dssm_model_l_user_semantic.pkl')
load(dssm_model.l_utt_semantic, '../weights/dssm_model_l_utt_semantic.pkl')

In [5]:
uid2msgs = {}

with codecs.open('../twitter_data/dssm_data_selected_users', encoding='utf8') as fin:
    for line in fin:
        id_, msg = line.strip().split('\t')
        if id_ in uid2msgs:
            uid2msgs[id_].append(msg)
        else:
            uid2msgs[id_] = [msg]
            
all_ids = sorted(map(int, uid2msgs.keys()))

In [6]:
def print_users_msgs(users=None):
    users = users or all_ids
    for id_ in users[:100]:
        print "******** EXAMPLE MESSAGES OF USER {} *********".format(id_)
        print
        for i in xrange(min(15, len(uid2msgs[str(id_)]))):
            print uid2msgs[str(id_)][i]
        print

In [7]:
print_users_msgs([11, 13, 24, 71, 74, 79, 80])

******** EXAMPLE MESSAGES OF USER 11 *********

со школьниками же такого не может быть)
))))) блин как мило боже))))))
ахаха "ебучий губернатор"))))
везука
ОЛЕГ НУ КАКОГО Ж ХРЕНА!!!!
беру!ХD
зарплату задерживают)) кот и так лучше меня кушает)))
ДА!)
ахахаха
просто кажусь наверное))
я не в теме вапще))
ну я там нигде не ржала надо сказать... наверное надо было слушать. я читала.
сомнительно ))
другая россия просто в ватный храм превратилась.
суббота же))))

******** EXAMPLE MESSAGES OF USER 13 *********

лишь бы привитая была
это точно ))
как выходной проведешь , на сопках ?:))
ну что дорогая, где там наши гостиницы ?:))
точно
это точно ))
и в театр тоже :))
норм, полет нормальный
Ленусик, мне пора:-)
ага, заезжай как нибудь :-)
доброе сударь ))
гыы
мерси и тебя сударь ))
это история, но я ее не осуждаю
ага

******** EXAMPLE MESSAGES OF USER 24 *********

Пасутся в Одноклассниках,а в Тви Каркают:)
Мячик Хоккеить!;)))
Согласен!!!!!!Гне они Все были?В дивизиях Вермахта...!!!!
Не правда!У 

In [6]:
dssm_model.similarity(13, 'та шо тебе надо')

0.075346462

In [21]:
dssm_model.similarity(80, 'шо тебе надо')

0.0

In [17]:
def find_most_similar(uid):
    u2msgs = uid2msgs.iteritems()
    id_msg_score = []
    for id_, msgs in tqdm(u2msgs):
        for msg in msgs:
            score = dssm_model.similarity(uid, msg)
            id_msg_score.append((id_, msg, score))
    return sorted(id_msg_score, key=lambda x: x[2], reverse=True)

In [ ]:
id_msg_score = find_most_similar(79)

1000it [47:39,  2.26s/it]


In [22]:
id_msg_score2 = find_most_similar(80)

1000it [50:03,  2.22s/it]


In [16]:
for id_, msg, score in id_msg_score[:200]:
    print "{}\t{:.2f}\t{}".format(id_, score, msg)

12	1.00	аа жаль
974	1.00	эмммм круто наверное
736	1.00	АА СОРЯН
443	1.00	базару нет
79	1.00	блин класс
37	1.00	хз че это
141	1.00	скучна
917	1.00	нет у нас завтра пары поставили
431	1.00	аааааа ясн
501	1.00	19 р было в дикси
256	1.00	нет щас секс приедет
621	1.00	плавали знаем за 200
39	1.00	ок а ты кто
230	1.00	нет зай
772	1.00	ЗАТКНИСЬ ПРОШУ НЕТ
516	1.00	нет мне одиноко без вас
446	1.00	заткнись говно
935	1.00	в моем возрасте она уже была замужем
458	1.00	и про мя
39	1.00	нит вы еще не готовы
810	1.00	ок спс
96	1.00	ок спс
10	1.00	ок спс
659	1.00	ок спс
659	1.00	ок спс
571	1.00	ок спс
462	1.00	ок спс
959	1.00	хорошенький епты
788	1.00	надо организовать пункт проката
593	1.00	нет моих зай Некоторые тут, а другие нет
495	1.00	блин кошмар
740	1.00	50 оттенков серого
571	1.00	нет спс
12	1.00	ааа ясн
12	1.00	ааа ясн
12	1.00	ааа ясн
571	1.00	сам ы лох
615	1.00	блин палево
643	1.00	Лиза заеб
266	1.00	ДА! обнял
266	1.00	да! Обнял
477	1.00	не обижай Киру(
45	1.00	“ там слишком опасно не ходи 

## Prepare data for DSSM training

In [25]:
with codecs.open('../twitter_data/yt_twitter_chains_3', encoding='utf8') as fin, codecs.open('../twitter_data/dssm_data_selected_users', 'w', encoding='utf8') as fout:
    fin.readline() # read "value" line
    for line in tqdm(fin):
        msgs = line.strip().split('\\t')
        last_msg = msgs[-1]
        uid = last_msg.split()[0]
        msg = ' '.join(last_msg.split()[1:])
        if uid in selected_uids:
            fout.write("{}\t{}\n".format(selected_uids[uid], msg))

10329968it [02:46, 62117.55it/s]


## Prepare data for finetuning on user `24203097`

In [9]:
def strip_uids(chain):
    """chain: "123 привет\t321 приветики!\t123 как делищи?"
    """
    msgs = chain.strip().split('\\t')
    msgs = [' '.join(msg.split()[1:]) for msg in msgs]
    return '\\t'.join(msgs)

In [45]:
UID = '24203097'
with codecs.open('../twitter_data/yt_twitter_chains_3', encoding='utf8') as fin, codecs.open('../twitter_data/user_{}_chains'.format(UID), 'w', encoding='utf8') as fout:
    fin.readline() # read "value" line
    for line in tqdm(fin):
        msgs = line.strip().split('\\t')
        last_msg = msgs[-1]
        uid = last_msg.split()[0]
        msg = ' '.join(last_msg.split()[1:])
        if uid == UID:
            fout.write(strip_uids(line)+'\n')

10329968it [02:57, 58358.47it/s]


## Prepare data for finetuning on all users using dssm score

In [46]:
with codecs.open('../twitter_data/yt_twitter_chains_3', encoding='utf8') as fin, codecs.open('../twitter_data/selected_user_chains', 'w', encoding='utf8') as fout:
    fin.readline() # read "value" line
    for line in tqdm(fin):
        uid = line.strip().split('\\t')[-1].split()[0]
        msg = strip_uids(line)
        if uid in selected_uids:
            fout.write(u"{}\t{}\n".format(selected_uids[uid], msg))

10329968it [04:21, 39460.93it/s]


In [ ]:
counter.most_common(1)

In [41]:
len(selected_chains)

788178